## Day 01

In [53]:
nums = map(
    x -> parse(Int, x),
    readlines(open("01.txt"))
)
@assert nums[end] == 10044

In [54]:
function window_sum(idx, window_size)
    sum(nums[idx - window_size + 1:idx])
end

function count_increasing(window_size)
    result = 0
    for idx in window_size + 1:length(nums)
        if window_sum(idx, window_size) > window_sum(idx - 1, window_size)
            result += 1
        end
    end
    result
end

count_increasing (generic function with 1 method)

In [55]:
println("easy: ", count_increasing(1))
println("hard: ", count_increasing(3))

easy: 1715
hard: 1739
